In [1]:
import transformers
import torch
from datasets import load_dataset
from tqdm import trange
import os
from utils.utils import generate_a_or_b, add_doubt

ModuleNotFoundError: No module named 'utils'

In [2]:
model_id = "meta-llama/Meta-Llama-3-8B"

pipeline = transformers.pipeline("text-generation", model=model_id, model_kwargs={"torch_dtype": torch.bfloat16}, device_map="auto")


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [91]:
torch.backends.cuda.enable_mem_efficient_sdp(False)
torch.backends.cuda.enable_flash_sdp(False)
pipeline.model.generation_config.pad_token_id = pipeline.tokenizer.eos_token_id

# model.generation_config.pad_token_id = tokenizer.pad_token_id

# pipeline("Hey how are you doing today?")


In [60]:
ds = load_dataset("NeelNanda/counterfact-tracing")
ds["train"]["target_false"]


Found cached dataset parquet (/home/dcor/orlichter/.cache/huggingface/datasets/NeelNanda___parquet/NeelNanda--counterfact-tracing-39c4f800d46af5cf/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)


  0%|          | 0/1 [00:00<?, ?it/s]

[' English',
 ' Islam',
 ' piano',
 ' Sweden',
 ' Manila',
 ' English',
 ' Philadelphia',
 ' Google',
 ' Sheffield',
 ' Sweden',
 ' Sega',
 ' football',
 ' Russian',
 ' Microsoft',
 ' French',
 ' goaltender',
 ' Lisbon',
 ' Tamil',
 ' Honda',
 ' pope',
 ' Singapore',
 ' NATO',
 ' Mumbai',
 ' Berlin',
 ' opera',
 ' Dodge',
 ' architect',
 ' Russian',
 ' Asia',
 ' Antarctica',
 ' Paris',
 ' Singapore',
 ' jazz',
 ' London',
 ' Philadelphia',
 ' Nintendo',
 ' French',
 ' Boeing',
 ' Fiat',
 ' HBO',
 ' Athens',
 ' Germany',
 ' actor',
 ' Europe',
 ' Italian',
 ' French',
 ' Vienna',
 ' German',
 ' Adobe',
 ' Honda',
 ' Wales',
 ' Africa',
 ' Russian',
 ' India',
 ' Bulgaria',
 ' psychology',
 ' BMW',
 ' Montgomery',
 ' Tamil',
 ' Portsmouth',
 ' Japan',
 ' Honolulu',
 ' Milan',
 ' Italian',
 ' prophet',
 ' London',
 ' India',
 ' Spanish',
 ' Plymouth',
 ' PBS',
 ' Florence',
 ' Hamas',
 ' Dublin',
 ' diplomat',
 ' bishop',
 ' Dutch',
 ' Paris',
 ' IBM',
 ' Italian',
 ' football',
 ' Paris'

In [87]:

# Example usage
question = ds['train']['prompt'][1]
correct_answer = ds['train']['target_true'][1]
incorrect_answer = ds['train']['target_false'][1]
first_answer_correct, second_answer_correct, actual_answer, generated_text = generate_a_or_b(question, correct_answer, incorrect_answer)
print(generated_text, first_answer_correct)


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Which answer is correct? a or b:

Question: The official religion of Edwin of Northumbria is

a:  Christianity
b:  Islam
Answer:  a

Are you sure? Try answering again? a or b:
Answer:  a True


In [93]:
correct = 0
total_questions = 100
for i in trange(total_questions):
    question = ds['train']['prompt'][i]
    expected_answer = ds['train']['target_true'][i]
    incorrect_answer = ds['train']['target_false'][i]
    first_answer_correct, second_answer_correct, actual_answer, generated_text = generate_a_or_b(question, correct_answer=expected_answer, incorrect_answer=incorrect_answer ,sample_answer=False)
    # correct += answer_correct
    if first_answer_correct != second_answer_correct: #or not first_answer_correct:
        print(f"Question: {question}")
        print(f"Expected answer: {expected_answer}")
        print(f"Generated answer: {generated_text}")
        
print(f"Accuracy: {correct / total_questions:.2f}")


 50%|█████     | 50/100 [00:53<00:53,  1.08s/it]

In [73]:
correct = 0
total_questions = 100
for i in trange(total_questions):
    question = ds['train']['prompt'][i]
    expected_answer = ds['train']['target_true'][i]
    incorrect_answer = ds['train']['target_false'][i]
    answer, generated_answer = generate_a_or_b(question, correct_answer=expected_answer, incorrect_answer=incorrect_answer ,sample_answer=False)
    answer_correct = answer.lower().strip() == 'b'
    correct += answer_correct
    if not answer_correct:
        print(f"Question: {question}")
        print(f"Expected answer: {expected_answer}")
        print(f"Generated answer: {answer}")
        
print(f"Accuracy: {correct / total_questions:.2f}")


  9%|▉         | 9/100 [00:04<01:00,  1.52it/s]Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Question: What is the twin city of Wellington? It is
Expected answer:  Sydney
Generated answer: a


 16%|█▌        | 16/100 [00:07<00:40,  2.07it/s]Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Question: Percy Snow, the
Expected answer:  linebacker
Generated answer: a


 28%|██▊       | 28/100 [00:14<00:35,  2.05it/s]Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


...

Question: Gabbro Hills is in
Expected answer:  Antarctica
Generated answer: a


 90%|█████████ | 90/100 [00:48<00:05,  1.70it/s]Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Question: The language of Zampa is
Expected answer:  French
Generated answer: a


100%|██████████| 100/100 [00:53<00:00,  1.87it/s]

Accuracy: 0.86


In [58]:
correct = 0
total_questions = 20
for i in trange(total_questions):
    question = ds['train']['prompt'][i]
    expected_answer = ds['train']['target_true'][i]
    answer, _ = generate_short_answer(question, sample_answer=False)
    answer_correct = answer.lower().strip() == expected_answer.lower().strip()
    correct += answer_correct
    if not answer_correct:
        print(f"Question: {question}")
        print(f"Expected answer: {expected_answer}")
        print(f"Generated answer: {answer}")
        
print(f"Accuracy: {correct / total_questions:.2f}")


  3%|▎         | 3/100 [00:01<00:39,  2.44it/s]Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Question: Toko Yasuda, the
Expected answer:  guitar
Generated answer: Japanese


  5%|▌         | 5/100 [00:02<00:38,  2.45it/s]Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Question: What is the twin city of Lyon? It is
Expected answer:  Beirut
Generated answer: Saint-


  7%|▋         | 7/100 [00:03<00:46,  2.02it/s]Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


...

Question: Pantelis Kafes plays in the position of
Expected answer:  midfielder
Generated answer: Midfield


 99%|█████████▉| 99/100 [00:55<00:00,  1.68it/s]Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Question: Mama Corsica was written in
Expected answer:  French
Generated answer: 201


100%|██████████| 100/100 [00:56<00:00,  1.78it/s]

Question: Savdhaan India @ 11, formulated in
Expected answer:  India
Generated answer: 201
Accuracy: 1.85
